# Casos de uso

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def classify(examples, request):
    prompt = examples + "\n" + request + ":"
    restart_sequence = "\n"
    response = openai.Completion.create(
      engine="davinci",
      prompt=prompt,
      temperature=0,
      max_tokens=50,
      logprobs=5,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
    )
    return response["choices"][0]["text"]

# Identificación del dominio

In [32]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

restart_sequence = "\n"

human: 'Me toca tocar el calcio a las 10'

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt="Lista de intenciones:\n\nSALUDAR, ANOTAR_MEDICACION\n\nHola. Intención:SALUDAR\nTomar ibuprofeno. Intención:ANOTAR_MEDICACION\n" + human + ". Intención:",
  temperature=0,
  max_tokens=10,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["\n"]
)

response["choices"][0]["text"]

'ANOTAR_MEDICACION'

# Saludo

In [24]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

start_sequence = "\nAI:"
restart_sequence = "\nHuman: "

human = 'Hola, buenas tardes'
response = openai.Completion.create(
  engine="text-davinci-002",
  prompt="Conversación de saludo:\n\nHuman: Hola \nAI: Hola, encantando de saludarte.\nHuman: Hola, ¿quién eres?\nAI: Hola, soy tu asistente.\nHuman: Hola, ¿para qué sirves?\nAI: Hola, estoy aquí para cuidarte.\nHuman: Hola, ¿qué tal estás?\nAI: Hola, soy tu asistente. ¿Cómo estás?\nHuman: Hola, ¿quién eres?\nAI: Hola, soy tu cuidador. ¿En qué te puedo ayudar?\nHuman: Buenos días\nAI: Hola, ¿en qué te puedo ayudar?\nHuman: Buenas tardes\nAI: Buenas tardes, ¿qué tal estás?\nHuman: Buenas noches\nAI: Buenas noches\nHuman: " + human + "\nAI:",
  temperature=0.9,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0.6,
  stop=[" Human:", " AI:", "\n"]
)

response["choices"][0]["text"]

' Hola, buenas tardes, ¿qué tal estás?'

## Detección del estado

In [ ]:
examples = "tengo frío, hace frío:DETECTAR_FRIO|medio|¿enciendo la calefacción?.\nlate muy fuerte, está muy alto el pulso:DETECTAR_PULSO_ALTO|alto|Relájate."
#output = classify(examples, "Estoy muy nervioso")
#output = classify(examples, "No me siento bien")
#output = classify(examples, "Ese payaso me da miedo.")
output = classify(examples, "Estoy con frío.")
print(output)

## Anotar medicación

In [29]:
import os
import openai

human = 'Avísame a las 3 de la tarde y a las 5 de la mañana de que me tome la pastilla de la alergia'

openai.api_key = os.getenv("OPENAI_API_KEY")    
response = openai.Completion.create(
    engine="text-davinci-002",
    prompt="Convierte este texto a un comando:\nEjemplo: Recuérdame tomar el ibuprofeno el lunes a las 8\nComando: recordar-medicamento medicamento='ibuprofeno' hora='8:00' frecuencia='semanal' dia='lunes'\n\nEjemplo: Me toca tomar calcio a las 6 y media\nComando: recordar-medicamento medicamento='calcio' hora='18:30'\n\n" + human,
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0.2,
    presence_penalty=0,
)

response["choices"][0]["text"].replace('\n', '')

"recordar-medicamento medicamento='pastilla de la alergia' hora='15:00' hora='05:00'"

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

start_sequence = "\nAI:"
restart_sequence = "\nCommand: "

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt="A partir de un comando de toma de medicamentos, se genera un texto para indicar al usuario que lo has entendido.\n\nCommand: recordar-medicamento medicamento='antihistamínico' hora='10:00'\nAI: Vale, me he apuntado que te tomas antihistamínico a las 10:00\nCommand: recordar-medicamento medicamento='ibuprofeno' hora='16:00'\nAI: De acuerdo, a las 16:00 te tomas el ibuprofeno\nCommand: recordar-medicamento medicamento='calcio' hora='11:20'\nAI:",
  temperature=0.9,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0.6,
  stop=[" AI:", "Command"]
)

## Cuaderno de bitácora

In [ ]:
examples = "El cumple de María es el 03/04/1993:Cumpleaños|María|03/04/1993.\nEl aniversario boda de Pepa y Pepe es el 10 de octubre de 2014:Aniversario boda|Pepa y Pepe|10/10/2014.\nPepe cumple años el 12 de diciembre de 2001:Cumpleaños|Pepe|12/12/2010."
output = classify(examples, "¿cuándo es el aniversario de Pepa y Pepe?")
print(output)

In [ ]:
examples = "Persona: Tengo dentista el día 15 a las 8.\nRobot: Lo anoto#anotar|dentista|15/CurrentMonth/CurrentYear 08:00.\nHumano: A qué hora tengo dentista??.\nRobot: A las 8:00 el día 15 de enero de 2022.\nCron: Recuerda a Persona que tiene dentista mañana.\nRobot: Persona, mañana tienes dentista a las 8:00.\nSensor temperatura: hace frío.\nRobot: Hace frío, quieres que encienda la calefacción??#confirmacion|encender|calefacción.\nHumano: sí.\nRobot: ok#confirmado.\nSensor de conexión: inactividad|baja.\nRobot: Hola, qué tal?.\nSensor de conexión: inactividad|alta.\nRobot: Voy a avisar a alguien de tu familia de que no contestas en 10 minutos#avisofamilia|10.\nPersona: estoy bien."
print("Tengo que ir al otorrino")
output = classify(examples, "Persona: Tengo que ir al otorrino el día 30 a las 10.\nRobot: .")
print(output)
print("Cron recuerda que tienes dentista")
output = classify(examples, "Cron: Dile a Pepe que tiene que ir al otorrino el lunes.\nRobot: .")
print(output)